
#Installation
---



In [ ]:
from pathlib import Path
import os
from datetime import date
import re
import glob

In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import box
import dask.dataframe as dd
pd.options.mode.chained_assignment = None

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
# Paths
p = Path('C:/Users/alexd/Dropbox/FFE')
pathShapefile = p / 'shapefile'
pathParquets = Path('F:/output')

print(len(os.listdir(pathParquets)))

In [ ]:
def read_and_concatenate_parquets(path=pathParquets):
    L = []
    files = pathParquets.glob('*scenario*')
    for file in files:
        print(f'{file} loaded file \n {len(L)}/2999')
        pqt = dd.read_parquet(file, engine='pyarrow')
        L.append(pqt)
    df = dd.concat(L)
    return df

    
def count_fid_occurences(df):
#     df.sort_values(by=['scenario'], inplace=True)
#     df.drop_duplicates(subset =['scenario', 'source'], inplace = True) 
    count_df = df['source'].value_counts().compute()
#     count_df.sort_values(by='scenario', inplace=True)
    count_df = pd.DataFrame({'source': count_df.index, 'count': count_df.values})
    count_df.to_parquet(str(pathShapefile) + '/' + f'CountBurn-{str(date.today())}.parquet', 
                        engine='auto', compression="GZIP")# could be datetime.now
    return count_df


def Merge(countDf,nameShapefile):
    gdfShape = gpd.read_file(pathShapefile / nameShapefile)
    gdfShape.insert(0, 'FID', range(0, len(gdfShape)))
    gdfShape.rename(columns={'FID': 'source'}, inplace=True)
    gdfShape = gdfShape[['source', 'geometry']]
    countDf.index.name = None
    merged = countDf.merge(gdfShape, on=['source'], how='left')
    return merged


def createShapefile(df):
    gdf = gpd.GeoDataFrame(df, geometry='geometry')
    gdf.to_file(os.path.join(str(pathShapefile) + "/" + "BurnscenarioWellington.shp"))
    return gdf

In [ ]:
test0 = pd.read_parquet('F:\output\scenario0_2020-08-06.parquet')

In [ ]:
test0.describe()

In [ ]:
%%time
concatDf = read_and_concatenate_parquets()

In [ ]:
concatDf.head()
concatDf.tail()

In [ ]:
%%time
countConcatDf = count_fid_occurences(concatDf)
countConcatDf

In [ ]:
%%time
mergedDf = Merge(countConcatDf, 'WellWHV_Buildings.shp')
countShape = createShapefile(mergedDf)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
# import contextily as ctx
%matplotlib inline
plt.style.use('seaborn-whitegrid')

FinnMeshblockShape = gpd.read_file(pathShapefile / 'Finn_MeshBlockSummary.shp')
countShape = gpd.read_file(pathShapefile / 'Burn3000scenarioWellington.shp')

fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, sharex=True, sharey=True, figsize=(12, 10))

FinnMeshblockShape.plot(ax=ax1, column='WellWHV_Bu', cmap='YlOrRd', alpha=0.7, legend=True)
# ctx.add_basemap(ax1, crs=2193)
countShape.plot(ax=ax2, column='count', cmap='YlOrRd', alpha=0.7, legend=True)
# ctx.add_basemap(ax2, crs=2193)

ax1.set_title('Original burn count at meshblock level')
ax1.ticklabel_format(useOffset=False, style='plain')
ax2.set_title('Network burn count at individual building level')
ax2.ticklabel_format(useOffset=False, style='plain')
ax1.tick_params(direction='out', length=6)
ax2.tick_params(direction="out", length=6)

fig.autofmt_xdate()
plt.tight_layout()
plt.savefig(pathShapefile / 'ComparisonGISvsNetwork_3000Burn.png', dpi=600)
plt.show()